# 4.1 Adding ETPs
We now only have a list of stocks. But we might also want to trade SPY, TLT or VXX. This section will append the <code>tickers_v2.csv</code> with a selection of ETPs to create <code>tickers_v3.csv</code>. Indices such as the VIX may also be of interest although Polygon does not have it. For these special cases, we will manually append them. We will add a prefix to non-tradeable tickers, so VIX and SPX become $VIX and $SPX. I am only interested in a selection of ETPs and indices; these were found by taking the most liquid ETF in its category at [etfdb.com](etfdb.com).

* Size ETFs: "SPY", "IJH", "IJR", 
    * Large-cap, Mid-cap, Small-cap, 
* Bond ETFs: "BIL", "SHY", "IEF", "TLT", "JPST", "VCSH", "VCIT", "VCLT" 
    * risk-free (1-3M, 1-3Y, 7-10Y, 20+Y), corporate (Overnight, 1-5Y, 5-10Y, 10-25Y)
* Sector ETFs: "XLE", "XLU", "XLK", "XLB", "XLP", "XLY", "XLC", "XLV", "XLF", "XLRE"
    * The sectors that Finviz also uses.
* Commodity ETFs "GLD", "SLV"
    * Gold, silver
* Volatility ETPs: "VXX", "UVIX", "SVIX", "SVXY"
    * Long, Long 2X, Short 1x, Short 0.5x (1x before and including 2018-02-27)

The only non-ETP index that I am interested in is the VIX. I will manually download daily VIX data from Yahoo and name it $VIX and set the type to INDEX. I could not find free 1-minute data. For simplicity I will just put this in the Polygon folder and not in a seperate Yahoo folder.

*Note: ETPs is an umbrella term for ETFs, ETVs and ETNs. However most investors just refer to them as 'ETFs'.*

In [1]:
#####
from polygon.rest import RESTClient
from datetime import date
from utils import get_tickers
import pandas as pd
import numpy as np

POLYGON_DATA_PATH = "../data/polygon/"

START_DATE = date(2019, 1, 1)
END_DATE = date(2023, 9, 1)

with open(POLYGON_DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

The ETFs can be added here manually.

In [2]:
###
size_ETFs = ["SPY", "IJH", "IJR"]
bond_ETFs = ["BIL", "SHY", "IEF", "TLT", "JPST", "VCSH", "VCIT", "VCLT" ]
sector_ETFs = ["XLE", "XLU", "XLK", "XLB", "XLP", "XLY", "XLC", "XLV", "XLF", "XLRE"]
commodity_ETFs = ["GLD", "SLV"]
volatility_ETFs = ["VXX", "UVIX", "SVIX", "SVXY"]
all_ETFs = size_ETFs + bond_ETFs + sector_ETFs + commodity_ETFs + volatility_ETFs

In [3]:
###
ticker_list_ETF = client.list_tickers(type="ETF", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)
ticker_list_ETN = client.list_tickers(type="ETN", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)
ticker_list_ETV = client.list_tickers(type="ETV", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)

ticker_list_ETF = pd.DataFrame(ticker_list_ETF)
ticker_list_ETN = pd.DataFrame(ticker_list_ETN)
ticker_list_ETV = pd.DataFrame(ticker_list_ETV)

tickers_ETP = pd.concat([ticker_list_ETF, ticker_list_ETN, ticker_list_ETV]).reset_index()
tickers_ETP = tickers_ETP[['ticker', 'name', 'active', 'cik', 'composite_figi', 'type']]
tickers_ETP = tickers_ETP[tickers_ETP["ticker"].isin(all_ETFs)]
tickers_ETP = tickers_ETP.reindex(columns = ["ID", "ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]) 
tickers_ETP.reset_index(drop=True, inplace=True)

tickers_ETP.head(3)

,ID,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,NaN,BIL,SPDR Bloomberg 1-3 Month T-Bill ETF,True,NaN,NaN,ETF,0001064642,BBG000RFQSH8
1,NaN,IEF,iShares 7-10 Year Treasury Bond ETF,True,NaN,NaN,ETF,0001100663,BBG000BHR296
2,NaN,IJH,iShares Core S&P Mid-Cap ETF,True,NaN,NaN,ETF,0001100663,BBG000BWC7P0


Here we will in the blanks using the <code>get_tickers_events</code> function. We need the start date from the ticker, which may not always be equal to START_DATE. I assume no delistings.

In [4]:
###
for index, row in tickers_ETP.copy().iterrows():
    if row["type"] == "ETF" or row["type"] == "ETN" or row["type"] == "ETV":
        try:
            tickers_ETP.iloc[index, tickers_ETP.columns.get_loc("start_date")] = max(date.fromisoformat(client.get_ticker_events(row["ticker"]).events[-1]['date']), START_DATE)
        except Exception as e:
            tickers_ETP.iloc[index, tickers_ETP.columns.get_loc("start_date")] = START_DATE
            print(row["ticker"])
            print(repr(e))

# Set remaining columns
tickers_ETP["end_date"].fillna(END_DATE, inplace=True)
tickers_ETP["ID"] = tickers_ETP["ticker"] + '-' + tickers_ETP["start_date"].astype(str)

Finally, manually add a ^VIX.

In [5]:
###
tickers_ETP.loc[len(tickers_ETP)] = f'$VIX-{START_DATE.isoformat()}', '$VIX', 'VIX Index', True, START_DATE, END_DATE, 'INDEX', np.NaN, np.NaN

We then add the ETPs to <code>tickers_v2</code> to get <code>tickers_v3</code>.

In [7]:
###
# Merge tickers_v2 and ETFs to get tickers_v3
tickers_v2 = get_tickers(v=2)
tickers_v3 = pd.concat([tickers_v2, tickers_ETP])
tickers_v3.reset_index(drop=True, inplace=True)
tickers_v3.to_csv("../data/tickers_v3.csv") 
tickers_v3.tail(3)

,ID,ticker,name,active,start_date,end_date,type,cik,composite_figi
8188,GLD-2019-01-01,GLD,"SPDR Gold Trust, SPDR Gold Shares",True,2019-01-01,2023-09-01,ETV,0001222333,BBG000CRF6Q8
8189,SLV-2019-01-01,SLV,iShares Silver Trust,True,2019-01-01,2023-09-01,ETV,0001330568,BBG000NDCRW7
8190,$VIX-2019-01-01,$VIX,VIX Index,True,2019-01-01,2023-09-01,INDEX,NaN,NaN
